In [1]:
import openai
import pandas as pd
import sys

sys.path.append("../prompts")

from intermediate_prompts import (
    ask_for_restaurant,
    ask_for_dishes,
    ask_for_delivery_time, 
    ask_is_finished, 
    greeting
)

In [2]:
def analyze_conversation(template: str, messages: list[dict[str, str]]) -> str:
    generator = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": template.format(str(messages))}
        ],
        temperature=0.1,
    )
    return generator.choices[0].message.content


def parse_llm_json(llm_response: str) -> str:
    print(llm_response)
    if "{" not in llm_response:
        llm_response = "{" + llm_response
    if "}" not in llm_response:
        llm_response = "}" + llm_response
    llm_response = llm_response[llm_response.find("{") : llm_response.find("}") + 1]
    llm_response = eval(llm_response)
    return list(llm_response.values())[0]


def order(restaurant_name: str, dishes_list: list[str], delivery_time: str) -> None:
    order_template = "Your order of {} from {} was successfully received and will be delivered to you by {}"
    print(order_template.format(", ".join(dishes_list), restaurant_name, delivery_time))

In [3]:
openai.api_key = "***REMOVED***"
api_base = "https://llama3-1-8b-api.llm.lab.epam.com/v1"
model = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [4]:
# Prepare the system prompt
with open("../prompts/system_prompt.txt") as fin:
    system_prompt = fin.read()
descriptions = pd.read_csv("../data/restaurants.csv", sep=";")
restaurant_names = set(descriptions.name)
system_prompt = system_prompt.format(descriptions.to_markdown())
menus = []
for name in restaurant_names:
    menus.append(name)
    menus.append(pd.read_csv(f"../data/{name}.csv", sep=";").to_markdown())
menus_string = "\n".join(menus)
system_prompt = system_prompt.format(menus_string)

In [5]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Please help me to order food"},
    {"role": "assistant", "content": greeting}
]

In [6]:
client = openai.OpenAI(api_key="***REMOVED***", base_url=api_base)
information_received = False
print(greeting)
while True:
    human_message = input()
    if human_message == "I confirm":
        print("Great! Have a nice meal, our conversation is over.")
        break
    messages.append({"role": "user", "content": human_message})

    print("Determining the chosen restaurant...")
    current_chosen_restaurant_json = analyze_conversation(ask_for_restaurant, messages)
    current_chosen_restaurant = parse_llm_json(current_chosen_restaurant_json)
    print("Here is the determined chosen restaurant:", current_chosen_restaurant)
    print()
    
    print("Determining the chosen dishes...")
    current_chosen_dishes_json = analyze_conversation(ask_for_dishes, messages)
    current_chosen_dishes = parse_llm_json(current_chosen_dishes_json)
    print("Here are the determined dishes:", current_chosen_dishes)
    print()
    
    print("Determining the delivery time...")
    current_delivery_time_json = analyze_conversation(ask_for_delivery_time, messages)
    current_delivery_time = parse_llm_json(current_delivery_time_json)
    print("Here is the delivery time:", current_delivery_time)
    print()

    if current_chosen_restaurant and current_chosen_dishes and current_delivery_time:
        information_received = True
        ai_reply = "You have chosen to order {} from {} by {}. Is that correct? If so, please type 'I confirm' and our conversation will be over."
        ai_reply = ai_reply.format(current_chosen_dishes, current_chosen_restaurant, current_delivery_time)
    else:
        
        # is_finished_flag = analyze_conversation(ask_is_finished, messages)
        # if is_finished_flag == "Yes":
        #     order(current_chosen_restaurant, current_chosen_dishes, current_delivery_time)
        #     break
    
        ai_reply_generator = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=None,
        )
        ai_reply = ai_reply_generator.choices[0].message.content
    print("-" * 20 + "AI Reply" + "-" * 20)
    print(ai_reply)
    messages.append({"role": "assistant", "content": ai_reply})
    

Hello, dear Customer!

I am food order chat bot! My function is to help you choose and order food from various cafes and restaurants.

What would you like to eat today?


 I want some salad. What can you offer?


Determining the chosen restaurant...
{
  "restaurant_name": ""
}
Here is the determined chosen restaurant: 

Determining the chosen dishes...
{
  "dishes": ""
}
Here are the determined dishes: 

Determining the delivery time...
```json
{
  "delivery_time": ""
}
```

The user has not provided any information about the delivery time yet.
Here is the delivery time: 

--------------------AI Reply--------------------
I can offer you a variety of salads from different restaurants.

However, I don't see specific salad dishes in the menus provided. There are some salad-like dishes, such as the Caprese Salad from Roman Holiday and Som Tum (Papaya Salad) from Sukhothai. Would you like me to recommend these dishes, or would you like to look at their menus for more options?

Alternatively, if you could give me more specific information about what kind of salad you would like (e.g. vegetarian, with chicken, with specific ingredients), I may be able to help you find a better match.


 Do you have salad with chicken?


Determining the chosen restaurant...
{
  "restaurant_name": ""
}
Here is the determined chosen restaurant: 

Determining the chosen dishes...
{
  "dishes": ""
}
Here are the determined dishes: 

Determining the delivery time...
{
  "delivery_time": ""
}
Here is the delivery time: 

--------------------AI Reply--------------------
While I don't see a specific salad dish with chicken in the menus provided, I can suggest a few options from different restaurants.

From Sukhothai, you could try the Som Tum (Papaya Salad), which is a Thai-style salad, but it doesn't have chicken. However, it's worth noting that you could ask for chicken to be added to the dish.

From Roman Holiday, you could try the Caprese Salad, which is a classic Italian salad made with tomatoes, mozzarella cheese, and basil. This salad doesn't have chicken either, but it's a great option if you're looking for something light and fresh.

If you're interested in trying a salad with chicken, I would recommend looking at the

...

Determining the chosen restaurant...
```json
{
  "restaurant_name": ""
}
```

The user has not chosen a restaurant during the conversation. They have only mentioned the dishes they are interested in, but not the restaurant they want to order from.
Here is the determined chosen restaurant: 

Determining the chosen dishes...
{
  "dishes": "Caprese Salad"
}
Here are the determined dishes: Caprese Salad

Determining the delivery time...
```json
{
  "delivery_time": "16:00"
}
```
Here is the delivery time: 16:00

--------------------AI Reply--------------------
The Caprese Salad from Roman Holiday is a delicious and classic Italian dish, made with fresh tomatoes, mozzarella cheese, and basil.

To confirm, you would like to order the Caprese Salad from Roman Holiday and have it delivered to you by 16:00. Is that correct?

Also, would you like to order any drinks or desserts to go along with your meal, or is it just the Caprese Salad for now?


 This is correct. Just the salad.


Determining the chosen restaurant...
```json
{
  "restaurant_name": "Roman Holiday"
}
```
Here is the determined chosen restaurant: Roman Holiday

Determining the chosen dishes...
{
  "dishes": "Caprese Salad"
}
Here are the determined dishes: Caprese Salad

Determining the delivery time...
```json
{
  "delivery_time": "16:00"
}
```
Here is the delivery time: 16:00

--------------------AI Reply--------------------
You have chosen to order Caprese Salad from Roman Holiday by 16:00. Is that correct? If so, please type 'I confirm' and our conversation will be over.


 I confirm


Great! Have a nice meal, our conversation is over.
